<a href="https://colab.research.google.com/github/khodizoda/sudoku_solver/blob/master/jamanetwork_com.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Set up the Environment and update**

Also get the base scraper class from Kevin and Simon

In [0]:
from google.colab import auth
auth.authenticate_user()

!curl https://raw.githubusercontent.com/Karocyt/covid-scraping/master/ArticleScraper.ipynb > ArticleScraper.ipynb
%run ArticleScraper.ipynb

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 19034  100 19034    0     0  49567      0 --:--:-- --:--:-- --:--:-- 49567
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpa

#Define jamanetwork.com scraper class

In [0]:
wd = init_selenium()
url_base = "https://jamanetwork.com/searchresults?"
url_end = "&page="

# how to add multiple search_keywords?
search_keywords = ['coronavirus']
url_search = ["q=" + keyword + "&f_ContentType=Article&exPrm_qqq=%7b!payloadDisMaxQParser+pf%3dTags+qf%3dTags%5e0.0000001+payloadFields%3dTags+bf%3d%7d%22" + keyword + "%22&exPrm_hl.q=" + keyword for c, keyword in enumerate(search_keywords)]
url_search = "".join(url_search)

dicts = []
for page_num in range(12):
  url = url_base + url_search + url_end + str(page_num+1)
  wd.get(url)
  articles = wd.find_elements_by_class_name('article--title')
  if len(articles) == 0: break
  for article in articles:
    dicts.append({
        'link': article.find_element_by_css_selector('a').get_attribute('href')
    })
print(url)
print("Total of " + str(len(dicts)) + " articles")
print(*dicts, sep='\n')

https://jamanetwork.com/searchresults?q=coronavirus&f_ContentType=Article&exPrm_qqq=%7b!payloadDisMaxQParser+pf%3dTags+qf%3dTags%5e0.0000001+payloadFields%3dTags+bf%3d%7d%22coronavirus%22&exPrm_hl.q=coronavirus&page=12
Total of 233 articles
{'link': 'https://jamanetwork.com/journals/jama/fullarticle/2763423?resultClick=1'}
{'link': 'https://jamanetwork.com/journals/jama/fullarticle/2761776?resultClick=1'}
{'link': 'https://jamanetwork.com/journals/jama/fullarticle/2759815?resultClick=1'}
{'link': 'https://jamanetwork.com/channels/health-forum/fullarticle/2761949?resultClick=1'}
{'link': 'https://jamanetwork.com/channels/health-forum/fullarticle/2760671?resultClick=1'}
{'link': 'https://jamanetwork.com/journals/jama/fullarticle/1764048?resultClick=1'}
{'link': 'https://jamanetwork.com/journals/jama/fullarticle/1679396?resultClick=1'}
{'link': 'https://jamanetwork.com/journals/jama/fullarticle/1456094?resultClick=1'}
{'link': 'https://jamanetwork.com/journals/jamaotolaryngology/fullartic

In [0]:
import re, io
import urllib.request

class JamanetworkSingleScraper(ArticleScraper):
  delim = "; "
  
  @property
  def title(self):
    return self.wd.find_element_by_xpath('//head/meta[@name="citation_title"]').get_attribute('content')

  # no output because no article_type in ArticleScraper
  @property
  def article_type(self):
    return self.wd.find_element_by_xpath('//div[@class="meta-article-type thm-col"]')

  @property
  def authors(self):
    author_elems = self.wd.find_elements_by_xpath('//head/meta[@name="citation_author"]')
    authors = [author.get_attribute('content') for author in author_elems]
    return self.delim.join(authors)

  @property
  def doi(self):
    return self.wd.find_element_by_xpath('//head/meta[@name="citation_doi"]').get_attribute('content')

  @property
  def abstract(self):
    try:
      return self.wd.find_element_by_xpath('//head/meta[@name="citation_abstract"]').get_attribute('content')
    except:
      return self.wd.find_element_by_xpath('//head/meta[@name="description"]').get_attribute('content')

  @property
  def date(self):
    try:
      return self.wd.find_element_by_xpath('//head/meta[@name="citation_publication_date"]').get_attribute('content')
    except:
      return self.wd.find_element_by_xpath('//head/meta[@name="citation_online_date"]').get_attribute('content')

  @property
  def body(self):
    body_elems = self.wd.find_elements_by_css_selector('p.para')
    paras = [para.text for para in body_elems]
    delim = "\n"
    return self.delim.join(paras)

  @property
  def source(self):
    return self.wd.find_element_by_xpath('//head/meta[@name="citation_journal_title"]').get_attribute('content')

  # source_impact_factor?
  
  @property
  def search_keyword(self):
    return self.delim.join(search_keywords)
  
  # Category?

  @property
  def license(self):
    try:
      return self.wd.find_element_by_id('PermissionsLink').get_attribute('href')
    except:
      return None
      
  @property
  def citations(self):
    try:
      return self.wd.find_element_by_xpath('//span[@class="artmet-number"]').text
    except:
      return 0

  @property
  def organization(self):
    organization_elems = self.wd.find_elements_by_xpath('//head/meta[@name="citation_author_institution"]')
    organizations = [org.get_attribute('content') for org in organization_elems]
    return self.delim.join(organizations)

  @property
  def keywords(self):
    keywords_elems = self.wd.find_elements_by_xpath('//head/meta[@name="citation_keyword"]')
    keys = [key.get_attribute('content') for key in keywords_elems]
    return self.delim.join(keys)

  @property
  def references(self):
    references_elems = self.wd.find_elements_by_xpath('//head/meta[@name="citation_reference"]')
    references = [ref.get_attribute('content') for ref in references_elems]
    return self.delim.join(references)

  @property
  def link(self):
    return self.url

  # link to pdf
  @property
  def extralinks(self):
    try:
      return self.wd.find_element_by_xpath('//head/meta[@name="citation_pdf_url"]').get_attribute('content')
    except:
      return None

In [0]:
import random, time
random.seed()


class JamanetworkScraper():
  def __init__(self):
    self.minDelay = 2
    self.maxDelayAdd = 0.1
    self.data = []

  def scrape(self, page_dicts):
    print("scraping " + str(len(page_dicts)) + " articles")
    for article in page_dicts:
      self.data.append(JamanetworkSingleScraper(article).parse(retry_sec=None))
      for k,v in self.data[-1].items():
        print(k, ':', v)
      print('\n')
      delay = self.minDelay + self.maxDelayAdd * random.random()
      print("sleeping for " + str(delay) + " seconds.")
      time.sleep(delay)
    return self.data

#Start scraping

In [0]:
scraper = JamanetworkScraper()
scraper.minDelay = 0.5
data = scraper.scrape(dicts)

Streaming output truncated to the last 5000 lines.
Authors : Chirag Bavishi; Thomas M. Maddox; Franz H. Messerli
DOI : 10.1001/jamacardio.2020.1282
Abstract : This Viewpoint discusses recommendations for angiotensin-converting enzyme inhibitor and angiotensin receptor blocker use in patients with or at risk of contrac
Date : 2020/04/03
Full body : T
he coronavirus disease 2019 (COVID-19) pandemic caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) has already surpassed the combined mortality inflicted by the severe acute respiratory syndrome (SARS) epidemic of 2002 and 2003 and the Middle East respiratory syndrome (MERS) epidemic of 2013. The pandemic is spreading at an exponential rate, with millions of people across the globe at risk of contracting SARS-CoV-2. Initial reports suggest that hypertension, diabetes, and cardiovascular diseases were the most frequent comorbidities in affected patients, and case fatality rates tended to be high in these individuals. In t

# Dump data to the bucket

In [0]:
import json

out_file = "jamanetwork.com.json"
with open(out_file, "w") as f:
  json.dump(data, f, indent=2)

!gsutil cp $out_file gs://middleware-bucket/JAMA/
!gsutil ls gs://middleware-bucket/JAMA/

Copying file://jamanetwork.com.json [Content-Type=application/json]...
-
Operation completed over 1 objects/1.4 MiB.                                      
gs://middleware-bucket/JAMA/jamanetwork.com.json


In [0]:
# Upload a readme file
from google.colab import files

filename = "jamanetwork.com.txt"
file = files.upload()[filename]

!gsutil cp $filename gs://middleware-bucket/JAMA/README.md

Saving jamanetwork.com.txt to jamanetwork.com (2).txt
Copying file://jamanetwork.com.txt [Content-Type=text/plain]...
/ [1 files][  755.0 B/  755.0 B]                                                
Operation completed over 1 objects/755.0 B.                                      
